In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, module_path)

import note.base as base
sys.path.pop(0)

SEED = 42
base.fix_seed(SEED)

2024-10-05 06:55:18.981024: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-05 06:55:19.002215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-05 06:55:19.024595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-05 06:55:19.030930: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-05 06:55:19.051990: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import pandas as pd

In [3]:
import re
import zipfile


def make_ms_file(df_select, output_file_path):
    df_select = df_select.reset_index(drop=True)
    invalid_chars = r'[\\/:*?"<>|]'

    for i in range(len(df_select)):
        formula = df_select['Formula'][i]
        parentmass = df_select['AverageMz'][i]
        ionization = df_select['Adducttype'][i]
        
        ms2_data = df_select['MSMSspectrum'][i]
        elements = ms2_data.split()  
        id = df_select.index.tolist()[i]
        metabolite = re.sub(invalid_chars, '_', df_select['Metabolitename'][i])
        ontology = re.sub(invalid_chars, '_', df_select['Ontology'][i])
        name_ont = f"{id}_{metabolite}_{ontology}"

        ms2_split_data = []
        for element in elements:
            parts = element.split(':')
            ms2_split_data.append(f"{parts[0]} {parts[1]}")

        output_file = f"{output_file_path}/{name_ont}.ms"
        with open(output_file, 'w', newline='') as file:
            
            file.write(f">compound {metabolite}_{ontology}\n")
            file.write(f">formula {formula}\n")
            file.write(f">parentmass {parentmass}\n")
            file.write(f">ionization {ionization}\n")
            
            file.write(">ms2\n")
            file.write('\n'.join(ms2_split_data))

def make_zipfile(folder_path, zip_filename):
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for foldername, subfolders, filenames in os.walk(folder_path):
            for filename in filenames:
                if filename.endswith('.ms'):
                    file_path = os.path.join(foldername, filename)
                    arcname = os.path.relpath(file_path, folder_path)
                    zipf.write(file_path, arcname)

pos

In [206]:
df_pos_p = pd.read_csv('../../data/canopus/tabledata_all_pos.csv').drop_duplicates()
df_pos = pd.read_csv('../../data/basedata/df_pos.csv').drop(columns=['Unnamed: 0']).drop_duplicates()

sel_columns = ['MSMSspectrum', 'Metabolitename', 'AlignmentID', 'indexid']
columns = ['MSMSspectrum', 'Metabolitename', 'AlignmentID']
df_pos = df_pos_p[sel_columns].merge(df_pos, on=columns, how='inner').drop_duplicates().sort_values('indexid', ascending=True)
df_pos.to_csv('../../data/canopus/df_pos_indexid.csv', index=False)

make ".ms" file

In [4]:
output_file_path = '../../data/canopus/ms2_spectrum_ms_pos'
make_ms_file(df_pos, output_file_path)

zip_filename = '../../data/canopus/ms2_spectrum_ms_pos.zip'
make_zipfile(output_file_path, zip_filename)

neg

In [260]:
df_neg_p = pd.read_csv('../../data/canopus/tabledata_all_neg.csv').drop_duplicates()
df_neg = pd.read_csv('../../data/basedata/df_neg.csv').drop(columns=['Unnamed: 0']).query('dataset != 77').drop_duplicates()

sel_columns = ['MSMSspectrum', 'Metabolitename', 'AlignmentID', 'sampleid','indexid']
columns = ['MSMSspectrum', 'Metabolitename', 'AlignmentID', 'sampleid']
df_neg = df_neg_p[sel_columns].merge(df_neg, on=columns, how='inner').drop_duplicates().sort_values('indexid', ascending=True)
df_neg.to_csv('../../data/canopus/df_neg_indexid.csv', index=False)

In [262]:
output_file_path = '../../data/canopus/ms2_spectrum_ms_neg'
make_ms_file(df_neg, output_file_path)

zip_filename = '../../data/canopus/ms2_spectrum_ms_neg.zip'
make_zipfile(output_file_path, zip_filename)